## Browsing HSC Footprints

**Last verified to run:** 2019-03-29

**Verified science platform release or release candidate:** 17.0.1

This notebook tests new functionality in `lsst.display.firefly` for browsing catalogs, footprints and images.

In [ ]:
import numpy as np

In [ ]:
from lsst.daf.persistence import Butler
import lsst.afw.geom as afwGeom

In [ ]:
import lsst.afw.display as afwDisplay

This example uses HSC-reprocessed data from `/datasets/hsc/repo/rerun/RC/w_2018_38/DM-15690/deepCoadd-results`

In [ ]:
butler = Butler('/datasets/hsc/repo/rerun/RC/w_2018_38/DM-15690/')

Define an ID for retrieving data

In [ ]:
dataId = dict(filter='HSC-R', tract=9813, patch='4,4')

Define a bounding boxes for a region of interest, one for the catalog and a larger one for the image.

In [ ]:
footprintsBbox = afwGeom.Box2I(corner=afwGeom.Point2I(16900, 18700),
                     dimensions=afwGeom.Extent2I(600,600))
imageBbox = afwGeom.Box2I(corner=afwGeom.Point2I(16800, 18600),
                     dimensions=afwGeom.Extent2I(800,800))

Retrieve a cutout for the corresponding exposure / coadd

In [ ]:
calexp = butler.get('deepCoadd_calexp_sub', dataId=dataId, bbox=imageBbox)

Retrieve a catalog for the entire region

In [ ]:
measCat = butler.get('deepCoadd_meas', dataId=dataId)

Select only those records with pixel locations inside the footprints bounding box

In [ ]:
catSelect = np.array([footprintsBbox.contains(afwGeom.Point2I(r.getX(), r.getY()))
                       for r in measCat])
catalogSubset = measCat.subset(catSelect)

Set up the Firefly display

In [ ]:
display1 = afwDisplay.Display(frame=1, backend='firefly')

In [ ]:
display1.setMaskTransparency(80)

In [ ]:
display1.scale('asinh', 10, 80, unit='percent', Q=6)

The `resetLayout` method sets up the image for upper left and table below, with space for plots at upper right

In [ ]:
display1.resetLayout()

Display the image

In [ ]:
display1.mtv(calexp)

Overlay the footprints and accompanying table.

Colors can be specified as a name like `'cyan'` or `afwDisplay.RED`; as an rgb value such as `'rgb(80,100,220)'`; or as rgb plus alpha (opacity) such as `'rgba('74,144,226,0.60)'`.

The `layerString` and `titleString` are concatenated with the frame, to make the footprint drawing layer name and the table title, respectively. If multiple footprint layers are desired, be sure to use different values of `layerString`.

In [ ]:
display1.overlayFootprints(catalogSubset, color='rgba(74,144,226,0.50)',
                           highlightColor='yellow', selectColor='orange',
                           style='outline', layerString='detection footprints ',
                           titleString='catalog footprints ')